In [1]:
import os
import shutil
import random
import json
import yaml

In [2]:
source_folder = 'output_final'
output_folder = 'datasets'
train_folder = os.path.join(output_folder, 'train')
val_folder = os.path.join(output_folder, 'val')
test_folder = os.path.join(output_folder, 'test')

In [3]:
# Crear carpetas de salida
for folder in [train_folder, val_folder, test_folder]:
    os.makedirs(folder, exist_ok=True)

# Obtener lista de archivos de imágenes y JSON
image_files = [f for f in os.listdir(source_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
json_files = [f for f in os.listdir(source_folder) if f.endswith('.json')]

# Leer clases desde los JSON
classes = set()
for json_file in json_files:
    with open(os.path.join(source_folder, json_file), 'r') as file:
        data = json.load(file)
        for category in data.get('categories', []):
            classes.add(category['name'])

In [4]:
# Ordenar las clases y asignar un índice
classes = sorted(classes)
class_to_index = {name: idx for idx, name in enumerate(classes)}

# Dividir las imágenes en 70% entrenamiento, 15% validación y 15% prueba
random.shuffle(image_files)
train_split = int(len(image_files) * 0.7)
val_split = int(len(image_files) * 0.85)

train_files = image_files[:train_split]
val_files = image_files[train_split:val_split]
test_files = image_files[val_split:]

In [5]:
# Función para mover archivos de imagen y sus JSON asociados
def move_files(files, dest_folder):
    for file_name in files:
        base_name = os.path.splitext(file_name)[0]
        image_path = os.path.join(source_folder, file_name)
        json_path = os.path.join(source_folder, base_name + '.json')

        # Mover la imagen al destino
        shutil.copy(image_path, os.path.join(dest_folder, file_name))

        # Mover el archivo JSON si existe
        if os.path.exists(json_path):
            shutil.copy(json_path, os.path.join(dest_folder, base_name + '.json'))

In [6]:
# Mover archivos a las carpetas correspondientes
move_files(train_files, train_folder)
move_files(val_files, val_folder)
move_files(test_files, test_folder)

print(f"Archivos distribuidos en {train_folder}, {val_folder}, y {test_folder}")
print(f"Clases detectadas: {classes}")

Archivos distribuidos en datasets\train, datasets\val, y datasets\test
Clases detectadas: ['Biche', 'Maduro', 'Rojo', 'biche', 'rojo']


In [7]:
# Crear el archivo YAML para YOLO
yaml_content = {
    'path': f"/content/{output_folder}",
    'train': 'train',
    'val': 'val',
    'test': 'test',
    'nc': len(classes),  # Número de clases
    'names': classes  # Lista de nombres de las clases
}

yaml_path = os.path.join(output_folder, 'dataset_config.yaml')
with open(yaml_path, 'w') as yaml_file:
    yaml.dump(yaml_content, yaml_file, default_flow_style=False)

print(f"Archivo YAML creado en {yaml_path}")

Archivo YAML creado en datasets\dataset_config.yaml


In [8]:
import os

def contar_imagenes(dataset_path):
    conjuntos = ['train', 'val', 'test']
    resumen = {}

    for conjunto in conjuntos:
        carpeta = os.path.join(dataset_path, conjunto)
        if os.path.exists(carpeta):
            num_imagenes = len([archivo for archivo in os.listdir(carpeta) if archivo.lower().endswith(('.png', '.jpg', '.jpeg'))])
            resumen[conjunto] = num_imagenes
        else:
            resumen[conjunto] = 0  # Si la carpeta no existe, se asigna 0

    # Mostrar el resumen de imágenes
    print("Resumen de imágenes en el dataset:")
    for conjunto, num_imagenes in resumen.items():
        print(f" - {conjunto.capitalize()}: {num_imagenes} imágenes")

    return resumen

# Ruta al directorio del dataset
ruta_dataset = 'datasets'
contar_imagenes(ruta_dataset)

Resumen de imágenes en el dataset:
 - Train: 44 imágenes
 - Val: 10 imágenes
 - Test: 10 imágenes


{'train': 44, 'val': 10, 'test': 10}